# Bounding Box Extraction Notebook  

## 📝 Purpose  
This notebook is a crucial step in the **handwriting recognition pipeline** that:  
1. Processes **word localization** results by extracting bounded word regions from images  
2. Prepares **labeled training data** for the handwriting recognition model  

## 🔄 Workflow  

### **Inputs**  
- **Label files**: Text files containing bounding box coordinates in YOLO format (normalized `[class, x_center, y_center, width, height]`)  
- **Source images**: Original images (e.g., scanned documents) with handwritten text  

### **Key Steps**  
1. **Reads bounding box data**:  
   - Parses label files to extract coordinates and class labels  
   - Maps bounding boxes to their corresponding images  
2. **Converts coordinates**:  
   - Transforms normalized YOLO coordinates → pixel coordinates  
   - Ensures bounding boxes stay within image boundaries (`max()`/`min()` checks)  
3. **Extracts word regions**:  
   - Crops images using bounding box coordinates  
   - Saves each word as a separate image  

### **Output Structure**  

In [1]:
import os
import cv2

# Function to create output directory if it doesn't exist
def create_output_directory(output_dir):
    os.makedirs(output_dir, exist_ok=True)

# Function to extract and save bounding box regions from images
def extract_and_save_bounding_boxes(image_dir, image_bounding_boxes, output_dir):
    # Iterate over each image filename in the image_bounding_boxes dictionary
    for image_filename, bounding_boxes_info in image_bounding_boxes.items():
        # Load the original image
        original_image = cv2.imread(os.path.join(image_dir, image_filename + '.jpg'))

        # Get the height and width of the original image
        height, width, _ = original_image.shape

        # Iterate over bounding box coordinates for the current image
        for line_number, bbox_info in enumerate(bounding_boxes_info, start=1):
            label, bbox = bbox_info
            # Extract bounding box coordinates
            x_center_norm, y_center_norm, width_norm, height_norm = bbox

            # Convert normalized coordinates to pixel coordinates
            x_center = int(x_center_norm * width)
            y_center = int(y_center_norm * height)
            width_pixels = int(width_norm * width)
            height_pixels = int(height_norm * height)

            # Calculate bounding box coordinates
            x1 = max(0, x_center - width_pixels // 2)
            y1 = max(0, y_center - height_pixels // 2)
            x2 = min(width, x_center + width_pixels // 2)
            y2 = min(height, y_center + height_pixels // 2)

            # Extract bounding box region from the original image
            bounding_box_image = original_image[y1:y2, x1:x2]

            # Create output directory if it doesn't exist
            output_folder = os.path.join(output_dir, image_filename)
            create_output_directory(output_folder)

            # Save the bounding box image
            output_filename = f"{label}_{line_number}.jpg"
            cv2.imwrite(os.path.join(output_folder, output_filename), bounding_box_image)

# Path to the directory containing the label files
label_dir = "C:\\Users\\Malek\\Desktop\\pfe\\test_separation"

# List all files in the directory
label_files = os.listdir(label_dir)

# Extract image names without extensions as dictionary keys
image_names = [os.path.splitext(filename)[0] for filename in label_files]

# Initialize the dictionary
image_bounding_boxes = {}

# Iterate through label files
for filename, image_name in zip(label_files, image_names):
    # Read bounding box coordinates from label file
    label_path = os.path.join(label_dir, filename)
    with open(label_path, "r") as label_file:
        # Extract and format bounding box coordinates
        bounding_boxes = [(line.split()[0], tuple(map(float, line.strip().split()[1:]))) for line in label_file]

    # Associate bounding box coordinates with image filename
    image_bounding_boxes[image_name] = bounding_boxes

# Path to the directory containing the images
image_dir = "C:\\Users\\Malek\\Desktop\\pfe\\img"

# Output directory to save the extracted bounding box regions
output_dir = "C:\\Users\\Malek\\Desktop\\pfe\\extracted_bounding_boxes"

# Extract and save bounding box regions from images
extract_and_save_bounding_boxes(image_dir, image_bounding_boxes, output_dir)
